# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn=pd.read_csv("https://www.stat4decision.com/telecom.csv")

In [3]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64
 17  Intl Calls      3333 non-null   i

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [14]:
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [4]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,350-1422,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [ ]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [5]:
# les méthodes de prétraitement
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# les outils de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

LabelEncoder va nous permettre de transformer les valeurs textuelles en entiers.
Nous pouvons utiliser pour chaque variable qualitative :

## Nouveauté : ColumnTransformer

On peut maintenant tranformer plusieurs colonnes simultanément

Essayez d'utliser cet outil :

In [7]:
from sklearn.compose import ColumnTransformer
# on utilisera un pipeline pour enchaîner les traitements
from sklearn.pipeline import Pipeline

In [8]:
# on définit les colonnes et les transformations pour 
# les colonnes quantitatives
col_quanti = ['Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls']

transfo_quanti = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='median')),
    ('standard', StandardScaler())])

# on définit les colonnes et les transformations pour
# les variables qualitatives
col_quali = ['Area Code', "Int'l Plan", 'VMail Plan']

transfo_quali = Pipeline(steps=[
    ('imputation', SimpleImputer(strategy='constant', fill_value='manquant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# on définit l'objet de la classe ColumnTransformer
# qui va permettre d'appliquer toutes les étapes
preparation = ColumnTransformer(
    transformers=[
        ('quanti', transfo_quanti , col_quanti),
        ('quali', transfo_quali , col_quali)])

In [9]:
churn_transfo = preparation.fit_transform(churn)

In [12]:
pd.DataFrame(churn_transfo)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.566767,0.476643,1.567036,-0.070610,-0.055940,-0.070427,0.866743,-0.465494,0.866029,-0.085008,-0.601195,-0.085690,-0.427932,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,-0.333738,1.124503,-0.334013,-0.108080,0.144867,-0.107549,1.058571,0.147825,1.059390,1.240482,-0.601195,1.241169,-0.427932,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,1.168304,0.675985,1.168464,-1.573383,0.496279,-1.573900,-0.756869,0.198935,-0.755571,0.703121,0.211534,0.697156,-1.188218,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,2.196596,-1.466936,2.196759,-2.742865,-0.608159,-2.743268,-0.078551,-0.567714,-0.078806,-1.303026,1.024263,-1.306401,0.332354,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,-0.240090,0.626149,-0.240041,-1.038932,1.098699,-1.037939,-0.276311,1.067803,-0.276562,-0.049184,-0.601195,-0.045885,1.092641,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,-0.432895,-1.167924,-0.433386,0.286348,1.299506,0.286880,1.547039,-0.874374,1.547188,-0.120832,0.617898,-0.125496,0.332354,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3329,0.942447,-2.164631,0.942714,-0.938353,-2.264816,-0.938172,-0.189297,1.170023,-0.188670,-0.228304,-0.194831,-0.231645,1.092641,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3330,0.018820,0.426808,0.019193,1.731930,-2.114211,1.732349,-0.177431,-0.465494,-0.175486,1.383778,0.617898,1.387123,0.332354,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3331,0.624778,0.227466,0.625153,-0.816080,-0.808966,-0.815203,-1.219628,1.885562,-1.221396,-1.876211,2.243356,-1.876950,0.332354,0.0,0.0,1.0,0.0,1.0,1.0,0.0


## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [15]:
y = churn["Churn?"]

In [16]:
x = churn_transfo

In [18]:
y.value_counts()

False.    2850
True.      483
Name: Churn?, dtype: int64

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [19]:
# on importe la fonction
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [20]:
x_train.shape

(2499, 20)

In [21]:
x_test.shape

(834, 20)

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle logistique
```python
from sklearn.linear_model import LogisticRegression
```

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [23]:
modele_rf = RandomForestClassifier(n_estimators=1000)
modele_knn = KNeighborsClassifier(n_neighbors=10)
modele_logit = LogisticRegression()

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [24]:
modele_rf.fit(x_train,y_train)
modele_knn.fit(x_train,y_train)
modele_logit.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [ ]:
modele_rf.feature_importances_

array([0.14395492, 0.0304749 , 0.14206942, 0.06955852, 0.03088603,
       0.06753035, 0.0406135 , 0.03099007, 0.03840635, 0.04800388,
       0.05600935, 0.042274  , 0.11749192, 0.00416674, 0.00587032,
       0.00435291, 0.04659702, 0.04309607, 0.01843011, 0.0192236 ])

Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [25]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
y_predict_logit = modele_logit.predict(x_test)

In [35]:
y_preict_rf = modele_rf.predict_proba(x_test)

In [36]:
y_preict_rf

array([[0.93 , 0.07 ],
       [0.925, 0.075],
       [0.975, 0.025],
       ...,
       [0.822, 0.178],
       [0.969, 0.031],
       [0.979, 0.021]])

In [32]:
(y_predict_rf == y_test).sum()/len(y_test)

0.9568345323741008

On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [33]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
accuracy_modele_logit = accuracy_score(y_test,y_predict_logit)
print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))
print("Pourcentage de bien classés pour le modèle logit :%.3f" %(accuracy_modele_logit))

Pourcentage de bien classés pour le modèle RF : 0.957
Pourcentage de bien classés pour le modèle kNN :0.886
Pourcentage de bien classés pour le modèle logit :0.862


In [ ]:
churn["Churn?"].value_counts(normalize=True)

False.    0.855086
True.     0.144914
Name: Churn?, dtype: float64

## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
confusion_matrix_logit=confusion_matrix(y_test,y_predict_logit)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
print("Matrice de confusion pour le modèle logit :",
confusion_matrix_logit, sep="\n")

Matrice de confusion pour le modèle RF :
[[714   8]
 [ 28  84]]
Matrice de confusion pour le modèle kNN :
[[720   2]
 [ 93  19]]
Matrice de confusion pour le modèle logit :
[[696  26]
 [ 89  23]]


## Le rappel (recall), la précision et le f1-score

Scikit-Learn possède des fonctions pour chacun de ces indicateurs, mais il peut
être intéressant d’utiliser une autre fonction qui les affiche pour chaque classe :

tp / (tp + fn)

In [ ]:
from sklearn.metrics import classification_report
print("Rapport pour le modèle RF :",
      classification_report(y_test,y_predict_rf) ,sep="\n")

Rapport pour le modèle RF :
              precision    recall  f1-score   support

      False.       0.95      1.00      0.97       725
       True.       0.97      0.66      0.79       109

    accuracy                           0.95       834
   macro avg       0.96      0.83      0.88       834
weighted avg       0.95      0.95      0.95       834



In [ ]:
print("Rapport pour le modèle kNN :",
      classification_report(y_test,y_predict_knn) ,sep="\n")

Rapport pour le modèle kNN :
              precision    recall  f1-score   support

      False.       0.90      0.99      0.94       725
       True.       0.79      0.30      0.44       109

    accuracy                           0.90       834
   macro avg       0.84      0.65      0.69       834
weighted avg       0.89      0.90      0.88       834



In [ ]:
print("Rapport pour le modèle logit :",
      classification_report(y_test,y_predict_logit) ,sep="\n")

Rapport pour le modèle logit :
              precision    recall  f1-score   support

      False.       0.89      0.96      0.92       725
       True.       0.43      0.18      0.26       109

    accuracy                           0.86       834
   macro avg       0.66      0.57      0.59       834
weighted avg       0.83      0.86      0.84       834



## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [ ]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])
auc_modele_logit=roc_auc_score(y_test,modele_logit.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
print("Aire sous la courbe ROC pour le modèle logit :" ,auc_modele_logit)

Aire sous la courbe ROC pour le modèle RF : 0.8931857007276178
Aire sous la courbe ROC pour le modèle kNN : 0.7796836444163239
Aire sous la courbe ROC pour le modèle logit : 0.773780449224929


## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [ ]:
import joblib
#joblib.dump(___, './data/modele_grid_pipe.pkl')

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [ ]:
import joblib
#___ = joblib.load('./data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
___.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.